In [1]:
import pandas as pd


In [3]:
df = pd.read_csv(f"logs_dataset (1).csv")

In [4]:
df.head()

,@timestamp,_id,ip_address
0,"July 8th 2019, 14:43:03.000",XswJ0msBoTGddM7vxMDB,10.1.1.285
1,"July 8th 2019, 14:43:01.000",dKQJ0msB7mP0GwVzvJjz,10.1.2.389
2,"July 8th 2019, 14:42:59.000",CcwJ0msBoTGddM7vtb8y,10.1.1.415
3,"July 8th 2019, 14:42:57.000",bKQJ0msB7mP0GwVzrZdT,10.1.1.79
4,"July 8th 2019, 14:42:55.000",L6QJ0msB7mP0GwVzpZeI,10.1.1.60


In [10]:
df['@timestamp'] = (df['@timestamp'].str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex = True) )

df['@timestamp'] = pd.to_datetime(
    df['@timestamp'], 
    format = '%B %d %Y, %H:%M:%S.%f'
)

In [12]:
print(df)

                @timestamp                   _id ip_address
721473 2019-06-09 00:06:09  DBuOOWsB7mP0GwVzhZ9U   10.1.1.1
720483 2019-06-09 01:28:39  bB7aOWsB7mP0GwVzDY5G   10.1.1.1
719233 2019-06-09 03:12:49  R0w5OmsBoTGddM7vayZT   10.1.1.1
719222 2019-06-09 03:13:45  U0w6OmsBoTGddM7vRi8R   10.1.1.1
718875 2019-06-09 03:42:39  z01UOmsBoTGddM7vuzyC   10.1.1.1
...                    ...                   ...        ...
3225   2019-07-08 11:49:00  3cBq0WsBoTGddM7va5TJ  10.1.2.99
2422   2019-07-08 12:22:29  QMKJ0WsBoTGddM7vE9N1  10.1.2.99
1704   2019-07-08 12:52:25  9pyk0WsB7mP0GwVze7sV  10.1.2.99
1138   2019-07-08 13:15:59  B8a60WsBoTGddM7vDnqQ  10.1.2.99
368    2019-07-08 13:47:19  M8jW0WsBoTGddM7vvplE  10.1.2.99

[721547 rows x 3 columns]


In [11]:
df.sort_values(['ip_address', '@timestamp'], inplace = True)

In [15]:
# df.groupby.value_count()
df['shift_time'] = df.groupby('ip_address')['@timestamp'].shift()


In [17]:
print(df.head())

                @timestamp                   _id ip_address  \
721473 2019-06-09 00:06:09  DBuOOWsB7mP0GwVzhZ9U   10.1.1.1   
720483 2019-06-09 01:28:39  bB7aOWsB7mP0GwVzDY5G   10.1.1.1   
719233 2019-06-09 03:12:49  R0w5OmsBoTGddM7vayZT   10.1.1.1   
719222 2019-06-09 03:13:45  U0w6OmsBoTGddM7vRi8R   10.1.1.1   
718875 2019-06-09 03:42:39  z01UOmsBoTGddM7vuzyC   10.1.1.1   

                shift_time  
721473                 NaT  
720483 2019-06-09 00:06:09  
719233 2019-06-09 01:28:39  
719222 2019-06-09 03:12:49  
718875 2019-06-09 03:13:45  


In [31]:
df['date'] = df['@timestamp'].dt.hour
df['weekday'] = df['@timestamp'].dt.weekday

In [32]:
df['is_weekend'] = ((df['weekday'] == 5) | (df['weekday'] == 6)).astype(int)

In [33]:
df.head()

,@timestamp,_id,ip_address,shift_time,data,weekday,is_weekend,hour,hour_bucket,date
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT,0,6,1,0,0,0
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09,1,6,1,1,0,1
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39,3,6,1,3,0,3
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49,3,6,1,3,0,3
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45,3,6,1,3,0,3


In [34]:
df['hour'] = df['@timestamp'].dt.hour
df['hour_bucket'] = df['hour']//4
# This integer division by 4, so we can analyse 4-hour buckets instead of every single hour

In [35]:
ip_addr ='ip_address'
ip_counts = df.groupby(ip_addr)['@timestamp'].count().reset_index()

In [36]:
ip_counts = ip_counts.rename(columns={'@timestamp':'total_count'})

In [37]:
daily_counts = df.groupby([ip_addr,'date'])['@timestamp'].count().reset_index()
daily_counts = daily_counts.rename(columns={'@timestamp':'daily_counts'})
daily_counts_avg = daily_counts.groupby(ip_addr).daily_counts.median().reset_index()

In [38]:
daily_counts_avg.head(5)

,ip_address,daily_counts
0,10.1.1.1,59.0
1,10.1.1.100,114.5
2,10.1.1.101,58.5
3,10.1.1.106,60.0
4,10.1.1.109,61.0


In [39]:
weekend_counts = df.groupby([ip_addr, 'is_weekend'])['@timestamp'].count().reset_index()
weekend_counts

,ip_address,is_weekend,@timestamp
0,10.1.1.1,0,975
1,10.1.1.1,1,471
2,10.1.1.100,0,1960
3,10.1.1.100,1,900
4,10.1.1.101,0,1006
...,...,...,...
767,10.1.2.90,1,871
768,10.1.2.95,0,1973
769,10.1.2.95,1,895
770,10.1.2.99,0,978
